This is a documention file for what I learnt from a Medium post. 
Below is the originl link:
https://medium.com/towards-data-science/optimization-newtons-method-profit-maximization-part-1-basic-optimization-theory-ff7c5f966565


# Section 1: Introduction

A review of first derivative (Gradient), second derivative (Hessian) and how we utilize the gradient or Newton method to get the optimal value that minimize/maximine the objecive function. 

Basically, there are two ways to do the optimization:
1. grid search iteration. easy to understand but may be conputational expensive 
2. calculate directly using calculus. calculate the Gradient of the objective function and set it to 0, then solve the optimal value. 

For complicated objective function, we have to derive both Hessian and Gradient and use Newton's way to get the optimal parameter. 

Below is the Python code to derive gradient, Hessian given an object function and use Newton's method to get the optimal values. 

Newton's method:

$$x1 = x0 - Hessisn^{-1} * Gradient $$

where x0 is the initial guess, it could be a single value or a vector of values, depend on the objective function

In [1]:
# import necessary libraries 
import sympy as sm
import numpy as np

In [2]:
# a function to derive first derivative(gradient) and calculate its value when x=x0 
def get_gradient(function, symbols, x0): # Add x0 as argument
    '''
    Helper function to solve for Gradient of SymPy function.
    '''
    d1 = {}
    gradient = np.array([])

    for s in symbols:
        d1[s]= sm.diff(function,s).evalf(subs=x0) # add evalf method
        gradient = np.append(gradient, d1[s])

    return gradient.astype(np.float64) # Change data type to float

# a function to derive second derivative(Hessian) and calculate its value when x=x0 
def get_hessian(function, symbols, x0): # Add x0 as argument
    '''
    Helper function to solve for Hessian of SymPy function.
    '''
    d2 = {}
    hessian = np.array([])

    for s1 in symbols:
        for s2 in symbols:
            d2[f"{s1}{s2}"] = sm.diff(function,s1,s2).evalf(subs=x0) # add evalf method
            hessian = np.append(hessian, d2[f"{s1}{s2}"])

    hessian = np.array(np.array_split(hessian,len(symbols)))

    return hessian.astype(np.float64) # Change data type to float

In [10]:
# Newton method to get the optimal value of x and y 
def newton_method(function,symbols,x0,iterations=100,mute=False):
    '''
    Function to run Newton's method to optimize SymPy function.
    '''
    # Dictionary of values to record each iteration
    x_star = {}
    x_star[0] = np.array(list(x0.values()))

    if not mute:
        print(f"Starting Values: {x_star[0]}")

    i=0
    while i < iterations:
        
        # Get gradient and hessian
        gradient = get_gradient(function, symbols, dict(zip(x0.keys(),x_star[i])))
        hessian = get_hessian(function, symbols, dict(zip(x0.keys(),x_star[i])))
        
        # Newton method iteration scheme
        x_star[i+1] = x_star[i].T - np.linalg.inv(hessian) @ gradient.T
        
        # Check convergence criteria
        if np.linalg.norm(x_star[i+1] - x_star[i]) < 10e-5:
            solution = dict(zip(x0.keys(),x_star[i+1]))
            print(f"\nConvergence Achieved ({i+1} iterations): Solution = {solution}")
            break 
        else: 
            solution = None
        
        if not mute:
            print(f"Step {i+1}: {x_star[i+1]}")
        
        i += 1
   
    return solution

In [14]:
# Define symbols & objective function
x, y = sm.symbols('x y')

# initialize the starting value of x and y, and it is totally randomo 
Gamma = [x,y]

# define the objective function
objective = 100*(y-x**2)**2 + (1-x)**2

In [8]:
# get the gradient when x= -1.2, y = 1.0 
get_gradient(objective, Gamma,{x:-1.2,y:1.0})

array([-215.6,  -88. ])

In [9]:
# get the Hessian when x= -1.2, y = 1.0 
get_hessian(objective, Gamma,{x:-1.2,y:1.0})

array([[1330.,  480.],
       [ 480.,  200.]])

In [11]:
newton_method(objective,Gamma,{x:-1.2,y:1.0},iterations=100,mute=False)

# after 6 interations, we get the optimal value of x and y that can minimize the objecctive function 

Starting Values: [-1.2  1. ]
Step 1: [-1.1752809   1.38067416]
Step 2: [ 0.76311487 -3.17503385]
Step 3: [0.76342968 0.58282478]
Step 4: [0.99999531 0.94402732]
Step 5: [0.9999957  0.99999139]

Convergence Achieved (6 iterations): Solution = {x: 0.9999999999999999, y: 0.9999999999814724}


{x: 0.9999999999999999, y: 0.9999999999814724}

# Part 2: Contrained Optimization

Constrained optimization is defined as “the process of optimizing an objective function with respect to some variables in the presence of constraints on those variables.

There are two types of contraints (equality and inequality)

## 2.1 Equality Constraint - The Lagrangian

$$ \mathcal{L} = f(x) + \sum_{j=1}^{r}\lambda_j h_j(x) $$

$\mathcal{L}$ represents the Lagrangian objective function, f(x) is the original objective function, $\sum_{j=1}^{r}\lambda_j h_j(x)$ is the sum of Lagrange multipliers to each constraint $j$

The goal is the find the optimal value of Xs,Lagrange multipliers that maximize or minimaze the Lagrangian objective function using Newton's method 

Example:
$$ min\;100(y-x^2)^2+(1-x)^2, $$
$$ s.t. x^2-y=2$$

then the Lagrangian function will be:
$$ \mathcal{L} = 100(y-x^2)^2+(1-x)^2+\lambda*(x^2-y-2)$$

In [15]:
x, y, λ  = sm.symbols('x y λ')

Langrangian_objective = 100*(y-x**2)**2 + (1-x)**2 + λ*(x**2-y-2)
Gamma = [x,y,λ]
Gamma0 = {x:-1.2,y:1,λ:1}

newton_method(Langrangian_objective,Gamma,Gamma0)

Starting Values: [-1.2  1.   1. ]
Step 1: [  -1.17555556   -0.61866667 -400.        ]
Step 2: [   0.7677616   -5.1870237 -400.       ]
Step 3: [   0.76806868   -1.4100706  -400.        ]
Step 4: [   0.99999563   -1.05379886 -400.        ]
Step 5: [   0.999996   -1.000008 -400.      ]

Convergence Achieved (6 iterations): Solution = {x: 0.9999999999999999, y: -1.0000000000160152, λ: -400.0}


{x: 0.9999999999999999, y: -1.0000000000160152, λ: -400.0}

## 2.2 Inequality Constraints - The Logarithmic Barrier Function 

$$\min_x\;f(x), x = [x_1,x_2,...,x_n] \in \mathbb{R}$$
$$ g_j(x) \le 0, j = 1,2,...,m  $$
f(x) is the objective functiono and g(x) is the contraint, we have m constraints

Interior-point methods/barrier methods 
$$ \mathcal{B}(x,\rho) = f(x) - \rho\sum_{j=1}^m\;log(c_j(x))$$
$$x = [x_1,x_2,...,x_n], c_j(x) = \left\{\begin{array}{rcl} g_j(x)&g_j(x)\ge0 \\ -g_j(x)&g_j(x)\le0\end{array}\right.$$
where $\rho$ is a small positive scalar, also known as the barrier parameter, $c(x)$ states that depending on constraints ($\ge 0 \;or \le 0 $) will dictate whether we use the negative or positive of that constraint. c(x) should be formulated to be always greater than 0.

How does it works?

As the optimal values approach the “barrier” outlined by the constraint, this method relies on the fact that the logarithmic function approaches negative infinity as the value approaches zero, thereby penalizing the objective function value. As ρ → 0, the penalization decreases and we converge to the solution. However, it is necessary to start with a sufficiently large ρ so that the penalization is large enough to prevent “jumping” out of the barriers. Therefore, the algorithm has one extra loop than Newton’s method alone — namely, we choose a starting value ρ, optimize the barrier function using Newton’s method, then update ρ by slowly decreasing it (ρ → 0), and repeat until convergence.

Example:
$$ min\;100(y-x^2)^2+(1-x)^2, $$
$$ s.t. x \le 0, y \ge 3$$

then the Barrier Lagrangian function will be:
$$ \mathcal{B} = 100(y-x^2)^2+(1-x)^2-\rho*log(((y-3)(-x)))$$


In [26]:
# update the function for inequality constraints  
def constrained_newton_method(function,symbols,x0,iterations=100,mute=False):
    '''
    Function to run Barrier method & Newton's method to optimize a
    constrained SymPy function.
    '''  
    
    # Values to record over each iteration
    x_star = {}
    x_star[0] = np.array(list(x0.values())[:-1])
    optimal_solutions = []
    optimal_solutions.append(dict(zip(list(x0.keys())[:-1],x_star[0])))
    
    # Barrier Method Iteration 
    step = 1 
    while step < iterations:

        # Starting rho
        if step == 1: 
            rho_sub = list(x0.values())[-1]

        # Evaluate function at rho value
        rho_sub_values = {list(x0.keys())[-1]:rho_sub}
        function_eval = function.evalf(subs=rho_sub_values)

        if not mute:
            print(f"Step {step} w/ {rho_sub_values}")
            print(f"Starting Values: {x_star[0]}")
        
        # Newton's Method
        i=0
        while i < iterations:
        
            gradient = get_gradient(function_eval, symbols[:-1], dict(zip(list(x0.keys())[:-1],x_star[i])))
            hessian = get_hessian(function_eval, symbols[:-1], dict(zip(list(x0.keys())[:-1],x_star[i])))

            x_star[i+1] = x_star[i].T - np.linalg.inv(hessian) @ gradient.T

            if np.linalg.norm(x_star[i+1] - x_star[i]) < 10e-5:
                solution = dict(zip(list(x0.keys())[:-1],x_star[i+1]))
                if not mute:
                    print(f"Convergence Achieved ({i+1} iterations): Solution = {solution}\n") 
                break
            
            i += 1

        # Record optimal solution for each barrier method iteration
        optimal_solution = x_star[i+1]
        previous_optimal_solution = list(optimal_solutions[step-1].values())
        optimal_solutions.append(dict(zip(list(x0.keys())[:-1],optimal_solution)))
        
        # Check for overall convergence
        if np.linalg.norm(optimal_solution - previous_optimal_solution) < 10e-5:
            print(f"\n Overall Convergence Achieved ({step} steps): Solution = {optimal_solutions[step]}\n")
            break

        # Set new starting point
        x_star = {}
        x_star[0] = optimal_solution

        # Update rho
        rho_sub = 0.9*rho_sub

        # Update Steps
        step += 1

    return optimal_solutions[step]

In [28]:
x, y, ρ = sm.symbols('x y ρ')

Barrier_objective = 100*(y-x**2)**2 + (1-x)**2 - ρ*sm.log((-x)*(y-3))
Gamma = [x,y,ρ] # Function requires last symbol to be ρ!
Gamma0 = {x:-15,y:15,ρ:10}  # set up the initial values 

#constrained_newton_method(Barrier_objective,Gamma,Gamma0)